You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of the importance of artificial intelligence in today's world
- Hook to capture the audience's attention

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing developments
- AI ethics and regulations

2. Key Players in the AI Industry:
- Google's DeepMind
- IBM Watson
- Amazon Web Services
- Microsoft Azure
- Tesla's Autopilot

3. Notewort

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a pivotal technology in today's world, revolutionizing various industries and shaping the future of society. From machine learning advancements to natural language processing developments, the impact of AI is undeniable. As we delve into the latest trends and key players in the AI industry, it is essential to understand the implications and opportunities that AI presents in our rapidly evolving digital landscape.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
Machine learning algorithms continue to evolve, enabling AI systems to learn from data, improve performance, and make predictions without being explicitly programmed. The advancements in machine learning have led to breakthroughs in various applications, from autonomous vehicles to personalized recommendations.

### Natural Lan

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a pivotal technology in today's world, revolutionizing various industries and shaping the future of society. From machine learning advancements to natural language processing developments, the impact of AI is undeniable. As we delve into the latest trends and key players in the AI industry, it is essential to understand the implications and opportunities that AI presents in our rapidly evolving digital landscape.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
Machine learning algorithms continue to evolve, enabling AI systems to learn from data, improve performance, and make predictions without being explicitly programmed. The advancements in machine learning have led to breakthroughs in various applications, from autonomous vehicles to personalized recommendations.

### Natural Language Processing Developments
Natural language processing (NLP) is a branch of AI that focuses on the interaction between computers and human language. Recent developments in NLP have enhanced language understanding, sentiment analysis, and language translation capabilities, paving the way for more sophisticated AI applications in communication and content analysis.

### AI Ethics and Regulations
As AI technologies become more prevalent in society, the need for ethical guidelines and regulations has become paramount. Issues such as bias in AI algorithms, data privacy concerns, and the impact of AI on job displacement have sparked discussions on how to ensure responsible AI development and deployment.

## Key Players in the AI Industry

### Google's DeepMind
Google's DeepMind is renowned for its achievements in AI research, particularly in the fields of reinforcement learning and neural networks. From defeating human champions in complex games to advancing healthcare diagnostics, DeepMind continues to push the boundaries of AI innovation.

### IBM Watson
IBM Watson is a cognitive computing system that leverages AI to process vast amounts of data, analyze complex information, and provide insights for decision-making. Watson's applications span industries such as healthcare, finance, and customer service, showcasing the power of AI in transforming business operations.

### Amazon Web Services
Amazon Web Services (AWS) offers a suite of AI services that enable businesses to integrate machine learning, natural language processing, and computer vision capabilities into their applications. With tools like Amazon SageMaker and Amazon Rekognition, AWS empowers organizations to harness the potential of AI for scalable and cost-effective solutions.

### Microsoft Azure
Microsoft Azure provides a comprehensive cloud platform that includes AI services for building, training, and deploying AI models. From cognitive services for language understanding to custom vision solutions, Azure's AI offerings cater to diverse industry needs, driving innovation and efficiency in AI development.

### Tesla's Autopilot
Tesla's Autopilot is a prime example of AI application in autonomous driving technology, utilizing machine learning algorithms to enable self-driving capabilities in Tesla vehicles. With features like traffic-aware cruise control and automatic lane-keeping, Autopilot showcases the potential of AI in enhancing safety and convenience on the roads.

## Noteworthy News in Artificial Intelligence

### Recent Breakthroughs in AI Research
The AI research community continues to achieve significant milestones, from developing more powerful AI models to exploring new frontiers in AI ethics and explainability. Recent breakthroughs in areas like generative adversarial networks (GANs) and transformer architectures have paved the way for cutting-edge AI applications and advancements.

### Impact of AI on Various Industries
AI is transforming industries across the board, from healthcare and finance to marketing and entertainment. By streamlining processes, improving decision-making, and unlocking new opportunities, AI is driving innovation and competitiveness in sectors that embrace its capabilities.

### AI Applications in Healthcare, Finance, and Marketing
In healthcare, AI is revolutionizing diagnostics, personalized treatment plans, and healthcare delivery systems. In finance, AI-powered algorithms are optimizing trading strategies, risk management, and fraud detection. In marketing, AI tools are enhancing customer segmentation, personalized recommendations, and campaign performance analysis, enabling businesses to engage audiences more effectively.

## Conclusion

As we navigate the ever-evolving landscape of AI technologies, staying informed about the latest trends and key players in the industry is essential for professionals, entrepreneurs, and AI enthusiasts. By understanding the potential of AI applications, addressing ethical considerations, and leveraging the expertise of leading AI companies, we can harness the power of AI to drive innovation, create value, and shape a better future for society.

## Call to Action: Stay informed about AI developments by subscribing to our newsletter

In conclusion, Artificial Intelligence is not just a technology but a transformative force that is reshaping how we live, work, and interact with the world. By exploring the latest trends, key players, and noteworthy news in the AI industry, we can gain valuable insights into the future of AI and its impact on society. Let us embrace the possibilities of AI with curiosity, responsibility, and a commitment to ethical innovation.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [19]:
topic = "DEVELOP AN INVESTMENT WALLET"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on DEVELOP AN INVESTMENT WALLET.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Develop an Investment Wallet

Introduction:
- Brief overview of what an investment wallet is and its importance in personal finance.
- Mention the benefits of having a well-developed investment wallet.

Key Points:
1. Latest Trends:
- Discuss the rise of robo-advisors and how they are changing the investment landscape.
- Explore the trend of sustainable and socially responsible investing.
- Highlight the increasing popularity of cryptocurrency as an investment option.

2. Key Players:
- Int

I now can give a great answer
Final Answer: 

# **Developing Your Investment Wallet: A Guide to Financial Independence**

In today's fast-paced world, where financial independence is a goal for many, having a well-developed investment wallet is crucial. An investment wallet refers to a collection of investment vehicles, such as stocks, bonds, and other assets, that individuals use to grow their wealth over time. It serves as a strategic tool to achieve long-term financial goals and secure a stable financial future. By diversifying your investments and staying informed about market trends, you can build a robust investment wallet that aligns with your financial objectives.

The investment landscape is constantly evolving, with new trends shaping the way individuals approach investing. Robo-advisors have gained popularity in recent years, offering automated investment solutions that cater to a wide range of investors. Sustainable and socially responsible investing is also on the rise, as

In [20]:
Markdown(result)

# **Developing Your Investment Wallet: A Guide to Financial Independence**

In today's fast-paced world, where financial independence is a goal for many, having a well-developed investment wallet is crucial. An investment wallet refers to a collection of investment vehicles, such as stocks, bonds, and other assets, that individuals use to grow their wealth over time. It serves as a strategic tool to achieve long-term financial goals and secure a stable financial future. By diversifying your investments and staying informed about market trends, you can build a robust investment wallet that aligns with your financial objectives.

The investment landscape is constantly evolving, with new trends shaping the way individuals approach investing. Robo-advisors have gained popularity in recent years, offering automated investment solutions that cater to a wide range of investors. Sustainable and socially responsible investing is also on the rise, as more individuals seek to align their investments with their values. Additionally, the growing interest in cryptocurrency as an investment option has captured the attention of many investors looking for alternative assets to diversify their portfolios.

When it comes to building your investment wallet, choosing the right players in the industry is paramount. Major investment firms and platforms like Vanguard, Betterment, and Robinhood offer a range of investment options and personalized services to help investors achieve their financial goals. It's essential to consider the role of financial advisors in guiding your investment decisions and selecting the ones that align with your investment objectives. Influential figures in the investment world, such as Warren Buffett and Cathie Wood, provide valuable insights and strategies that can inspire and inform your investment decisions.

Staying informed about recent market developments and regulatory changes is crucial for effective investment management. By keeping up to date with the latest news and trends, you can make informed decisions about your investment strategy and adjust your portfolio accordingly. Success stories of individuals who have effectively developed their investment wallets serve as inspiration and motivation for others looking to achieve financial independence through smart investing.

To build a successful investment wallet, it's essential to prioritize diversification and risk management. By spreading your investments across different asset classes and sectors, you can reduce the impact of market volatility and protect your portfolio from significant losses. Understanding your risk tolerance and investment goals is key to developing a well-balanced and resilient investment wallet that can weather market fluctuations and provide long-term growth.

Diversification is a fundamental principle of investing that involves spreading your investments across various assets to minimize risk. By diversifying your portfolio, you can reduce the impact of market fluctuations and protect your investments from significant losses. Risk management is equally important, as it involves assessing your risk tolerance and implementing strategies to mitigate potential risks. By incorporating diversification and risk management into your investment approach, you can build a strong and resilient investment wallet that can withstand market uncertainties and generate sustainable returns.

In conclusion, developing an investment wallet is a strategic approach to achieving financial independence and securing a stable financial future. By staying informed about the latest investment trends, choosing the right players in the industry, and prioritizing diversification and risk management, you can build a robust investment portfolio that aligns with your financial goals. Take the first step towards financial independence today by starting to build your investment wallet and taking control of your financial future. Remember, the journey to financial independence begins with a well-developed investment wallet that reflects your investment objectives and risk tolerance. Start building your investment portfolio today and empower yourself to achieve your long-term financial goals.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).